# **Homework 3 - Convolutional Neural Network**

This is the example code of homework 3 of the machine learning course by Prof. Hung-yi Lee.

In this homework, you are required to build a convolutional neural network for image classification, possibly with some advanced training tips.


There are three levels here:

**Easy**: Build a simple convolutional neural network as the baseline. (2 pts)

**Medium**: Design a better architecture or adopt different data augmentations to improve the performance. (2 pts)

**Hard**: Utilize provided unlabeled data to obtain better results. (2 pts)

In [ ]:
import gc
import torch
gc.collect()
torch.cuda.empty_cache()

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Feb 23 04:58:15 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--


## **About the Dataset**

The dataset used here is food-11, a collection of food images in 11 classes.

For the requirement in the homework, TAs slightly modified the data.
Please DO NOT access the original fully-labeled training data or testing labels.

Also, the modified dataset is for this course only, and any further distribution or commercial use is forbidden.

In [ ]:
"""
# Download the dataset
# You may choose where to download the data.

# Google Drive
"""
#!gdown --id '1awF7pZ9Dz7X1jn1_QAiKN-_v56veCEKy' --output food-11.zip
"""
# Dropbox
# !wget https://www.dropbox.com/s/m9q6273jl3djall/food-11.zip -O food-11.zip

# MEGA
# !sudo apt install megatools
# !megadl "https://mega.nz/#!zt1TTIhK!ZuMbg5ZjGWzWX1I6nEUbfjMZgCmAgeqJlwDkqdIryfg"

# Unzip the dataset.
# This may take some time.
!unzip -q food-11.zip
"""

'\n# Dropbox\n# !wget https://www.dropbox.com/s/m9q6273jl3djall/food-11.zip -O food-11.zip\n\n# MEGA\n# !sudo apt install megatools\n# !megadl "https://mega.nz/#!zt1TTIhK!ZuMbg5ZjGWzWX1I6nEUbfjMZgCmAgeqJlwDkqdIryfg"\n\n# Unzip the dataset.\n# This may take some time.\n!unzip -q food-11.zip\n'

## **Import Packages**

1.   [清單項目](https://)
2.   清單項目



First, we need to import packages that will be used later.

In this homework, we highly rely on **torchvision**, a library of PyTorch.

In [ ]:
# Import necessary packages.
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from PIL import Image
# "ConcatDataset" and "Subset" are possibly useful when doing semi-supervised learning.
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset
from torchvision.datasets import DatasetFolder

# This is for the progress bar.
#from tqdm.auto import tqdm #may have some bug
from tqdm import tqdm



## **Change Dir to google drive**

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/NTU_colab/hw3
#!gdown --id '1awF7pZ9Dz7X1jn1_QAiKN-_v56veCEKy' --output food-11.zip



Mounted at /content/drive
/content/drive/MyDrive/NTU_colab/hw3


## **Dataset, Data Loader, and Transforms**

Torchvision provides lots of useful utilities for image preprocessing, data wrapping as well as data augmentation.

Here, since our data are stored in folders by class labels, we can directly apply **torchvision.datasets.DatasetFolder** for wrapping data without much effort.

Please refer to [PyTorch official website](https://pytorch.org/vision/stable/transforms.html) for details about different transforms.

In [ ]:
# It is important to do data augmentation in training.
# However, not every augmentation is useful.
# Please think about what kind of augmentation is helpful for food recognition.

# Basic Transformation
train_tfm = transforms.Compose([
    # Resize the image into a fixed shape (height = width = 128)
    transforms.Resize((128, 128)),
    # You may add some transforms here.
    # ToTensor() should be the last one of the transforms.
    transforms.ToTensor(),
    #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Training transformation with horizontal flipping
train_tfm2 = transforms.Compose([
    # Resize the image into a fixed shape (height = width = 128)
    transforms.Resize((128, 128)),
    # ToTensor() should be the last one of the transforms.
    transforms.RandomHorizontalFlip(p=1.0),
    transforms.ToTensor(),
    #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

#Training transformation with random rotationn
train_tfm3 = transforms.Compose([
    # Resize the image into a fixed shape (height = width = 128)
    transforms.Resize((128, 128)),
    # ToTensor() should be the last one of the transforms.
    transforms.RandomRotation((30,180)),
    transforms.ToTensor(),
    #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

#Transformation with rnadom prospective change
train_tfm4 = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomPerspective(p=1.0),
    transforms.ToTensor(),
    #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_tfm5 = transforms.Compose([
    # Resize the image into a fixed shape (height = width = 128)
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(p=1.0),
    transforms.ToTensor(),
    #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Transformation with color jitter
train_tfm6 = transforms.Compose([
    # Resize the image into a fixed shape (height = width = 128)
    transforms.Resize((128, 128)),
    transforms.ColorJitter(brightness=(0, 5), contrast=(
    0, 5), saturation=(0, 5), hue=(-0.1, 0.1)),
    transforms.ToTensor(),
    #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# We don't need augmentations in testing and validation.
# All we need here is to resize the PIL image and transform it into Tensor.
test_tfm = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_transform = transforms.Compose([
    #     transforms.Resize((224, 224)),
    #     transforms.ToTensor(),
    transforms.ToPILImage(),
    # transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    autoaugment.AutoAugment(),
    transforms.ToTensor(),
    # transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

In [ ]:
# Batch size for training, validation, and testing.
# A greater batch size usually gives a more stable gradient.
# But the GPU memory is limited, so please adjust it carefully.
#!unzip food-11.zip
batch_size = 96

# Construct datasets.
# The argument "loader" tells how torchvision reads the data.
train_set1 = DatasetFolder("food-11/training/labeled", loader=lambda x: Image.open(x), extensions="jpg", transform=train_tfm)
train_set2 = DatasetFolder("food-11/training/labeled", loader=lambda x: Image.open(x), extensions="jpg", transform=train_tfm2)
train_set3 = DatasetFolder("food-11/training/labeled", loader=lambda x: Image.open(x), extensions="jpg", transform=train_tfm3)
train_set4 = DatasetFolder("food-11/training/labeled", loader=lambda x: Image.open(x), extensions="jpg", transform=train_tfm4)
train_set5 = DatasetFolder("food-11/training/labeled", loader=lambda x: Image.open(x), extensions="jpg", transform=train_tfm5)
train_set6 = DatasetFolder("food-11/training/labeled", loader=lambda x: Image.open(x), extensions="jpg", transform=train_tfm6)
# As the patch = "food-11/training/labeled/0, "food-11/training/labeled"/1, "food-11/training/labeled/2... it will automatically classify the class label by DatasetFolder"


train_set = ConcatDataset([train_set1, train_set2, train_set3, train_set4, train_set5, train_set6])
valid_set = DatasetFolder("food-11/validation", loader=lambda x: Image.open(x), extensions="jpg", transform=test_tfm)
#unlabeled_set = DatasetFolder("food-11/training/unlabeled", loader=lambda x: Image.open(x), extensions="jpg", transform=train_tfm)

unlabeled_set1 = DatasetFolder("food-11/training/unlabeled", loader=lambda x: Image.open(x), extensions="jpg", transform=train_tfm)
unlabeled_set2 = DatasetFolder("food-11/training/unlabeled", loader=lambda x: Image.open(x), extensions="jpg", transform=train_tfm2)
unlabeled_set3 = DatasetFolder("food-11/training/unlabeled", loader=lambda x: Image.open(x), extensions="jpg", transform=train_tfm3)
unlabeled_set4 = DatasetFolder("food-11/training/unlabeled", loader=lambda x: Image.open(x), extensions="jpg", transform=train_tfm4)
unlabeled_set5 = DatasetFolder("food-11/training/unlabeled", loader=lambda x: Image.open(x), extensions="jpg", transform=train_tfm5)
unlabeled_set6 = DatasetFolder("food-11/training/unlabeled", loader=lambda x: Image.open(x), extensions="jpg", transform=train_tfm6)
unlabeled_set = ConcatDataset([unlabeled_set1, unlabeled_set2, unlabeled_set3, unlabeled_set4, unlabeled_set5, unlabeled_set6])
#It will still assign 00 to the label of unlabeled_set as the path= "food-11/training/unlabeled/00"

test_set = DatasetFolder("food-11/testing", loader=lambda x: Image.open(x), extensions="jpg", transform=test_tfm)
#It will still assign 00 to the label of test_set as the path= "food-11/testing/00"

# Construct data loaders.
# pin_memory = false, data will be swapped to driver if necessary
# default num_workers = 8
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=16, pin_memory=True)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=16, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


## **Model**

The basic model here is simply a stack of convolutional layers followed by some fully-connected layers.

Since there are three channels for a color image (RGB), the input channels of the network must be three.
In each convolutional layer, typically the channels of inputs grow, while the height and width shrink (or remain unchanged, according to some hyperparameters like stride and padding).

Before fed into fully-connected layers, the feature map must be flattened into a single one-dimensional vector (for each image).
These features are then transformed by the fully-connected layers, and finally, we obtain the "logits" for each class.

### **WARNING -- You Must Know**
You are free to modify the model architecture here for further improvement.
However, if you want to use some well-known architectures such as ResNet50, please make sure **NOT** to load the pre-trained weights.
Using such pre-trained models is considered cheating and therefore you will be punished.
Similarly, it is your responsibility to make sure no pre-trained weights are used if you use **torch.hub** to load any modules.

For example, if you use ResNet-18 as your model:

model = torchvision.models.resnet18(pretrained=**False**) → This is fine.

model = torchvision.models.resnet18(pretrained=**True**)  → This is **NOT** allowed.

In [ ]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # The arguments for commonly used modules:
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        # out_channels = feature filter num, by adding 3 RGB input channel result within 1 filter
        # torch.nn.MaxPool2d(kernel_size, stride, padding)
        # after 2x2 kernel maxpool, result will be intput witdth/2 * height/2

        # input image size: [3, 128, 128]
        """" original
        self.cnn_layers = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),

            nn.Conv2d(64, 128, 3, 1, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),

            nn.Conv2d(128, 256, 3, 1, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(4, 4, 0),
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(256 * 8 * 8, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(p=0.5),
            nn.Linear(256, 11)
        )
        """
        # add 2 Conv layers
        self.cnn_layers = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),

            nn.Conv2d(64, 128, 3, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),

            nn.Conv2d(128, 256, 3, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),

            nn.Conv2d(256, 512, 3, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),

            nn.Conv2d(512, 1024, 3, 1),
            nn.BatchNorm2d(1024),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0)
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(4096, 1024),
            nn.ReLU(),
            nn.BatchNorm1d(1024),
            nn.Dropout(0.6),
            nn.Linear(1024, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.5),
            nn.Linear(128, 11)
        )

    def forward(self, x):
        # input (x): [batch_size, 3, 128, 128]
        # output: [batch_size, 11]

        # Extract features by convolutional layers.
        x = self.cnn_layers(x)

        # The extracted feature map must be flatten before going to fully-connected layers.
        x = x.flatten(1)

        # The features are transformed by fully-connected layers to obtain the final logits.
        x = self.fc_layers(x)
        return x


class block(nn.Module):
  def __init__(self,in_channels, out_channels, identity_downsample=None, stride=1):
    super().__init__()
    self.expansion=4
    self.conv1=nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0)
    self.bn1=nn.BatchNorm2d(out_channels)
    self.conv2=nn.Conv2d(out_channels, out_channels, kernel_size=3,stride=stride,padding=1)
    self.bn2=nn.BatchNorm2d(out_channels)
    self.conv3=nn.Conv2d(out_channels, out_channels*self.expansion,kernel_size=1,stride=1,padding=0)
    self.bn3=nn.BatchNorm2d(out_channels*self.expansion)
    self.relu=nn.ReLU()
    self.identity_downsample=identity_downsample

  def forward(self,x):
    identity=x
    x=self.conv1(x)
    x=self.bn1(x)
    x=self.relu(x)
    x=self.conv2(x)
    x=self.bn2(x)
    x=self.relu(x)
    x=self.conv3(x)
    x=self.bn3(x)

    if self.identity_downsample is not None:
      identity=self.identity_downsample(identity)
    x+=identity
    x=self.relu(x)
    return x

class ResNet(nn.Module): #[3,4,6,3]
  def __init__(self,block,layers,image_channels,num_classes):
    super(ResNet,self).__init__()
    self.in_channels=64
    self.conv1=nn.Conv2d(image_channels,64,kernel_size=7,stride=2,padding=3)
    self.bn1=nn.BatchNorm2d(64)
    self.relu=nn.ReLU()
    self.maxpool=nn.MaxPool2d(kernel_size=3,stride=2,padding=1)

    self.layers1=self.make_layer(block,layers[0],out_channels=64,stride=1)
    self.layers2=self.make_layer(block,layers[0],out_channels=128,stride=2)
    self.layers3=self.make_layer(block,layers[0],out_channels=256,stride=2)
    self.layers4=self.make_layer(block,layers[0],out_channels=512,stride=2)

    self.avgpool=nn.AdaptiveAvgPool2d((1,1))
    self.fc=nn.Linear(512*4,num_classes)

  def forward(self,x):
    x=self.conv1(x)
    x=self.bn1(x)
    x=self.relu(x)  # Fixed the relu call
    x=self.maxpool(x)

    x=self.layers1(x)  # Fixed the layer names
    x=self.layers2(x)  # Fixed the layer names
    x=self.layers3(x)  # You need to add this line
    x=self.layers4(x)  # You need to add this line and fix the name
    x=self.avgpool(x)

    x=x.reshape(x.shape[0],-1)
    x=self.fc(x)

    return x

  def make_layer(self,block,num_residual_blocks,out_channels,stride):
    identity_downsample=None
    layers=[]

    if stride!=1 or self.in_channels !=out_channels*4:
      identity_downsample= nn.Sequential(nn.Conv2d(self.in_channels, out_channels*4,kernel_size=1,
        stride=stride),nn.BatchNorm2d(out_channels*4))

    layers.append(block(self.in_channels, out_channels, identity_downsample, stride))
    self.in_channels= out_channels*4

    for i in range(num_residual_blocks-1):
      layers.append(block(self.in_channels, out_channels)) # 256->64*4(256) again

    return nn.Sequential(*layers)


def ResNet18(img_channels=3,num_classes=11):
  return ResNet(block, [2,2,2,2],img_channels,num_classes)

def ResNet50(img_channels=3, num_classes=1000):
  return ResNet(block, [3,4,6,3],img_channels,num_classes)

def ResNet101(img_channels=3, num_classes=1000):
  return ResNet(block, [3,4,23,3],img_channels,num_classes)

def ResNet152(img_channels=3, num_classes=1000):
  return ResNet(block, [3,8,36,3],img_channels,num_classes)


def test():
  net=ResNet18()
  x=torch.randn(2,3,224,224)
  y=net(x).to('cuda')
  print(y.shape)
test()


torch.Size([2, 11])


## **Training**

You can finish supervised learning by simply running the provided code without any modification.

The function "get_pseudo_labels" is used for semi-supervised learning.
It is expected to get better performance if you use unlabeled data for semi-supervised learning.
However, you have to implement the function on your own and need to adjust several hyperparameters manually.

For more details about semi-supervised learning, please refer to [Prof. Lee's slides](https://speech.ee.ntu.edu.tw/~tlkagk/courses/ML_2016/Lecture/semi%20(v3).pdf).

Again, please notice that utilizing external data (or pre-trained model) for training is **prohibited**.

In [ ]:
#If it is a multi-label classification tasks
# Convert your true_labels to the correct dtype (float) and device as your model's output
"""
true_labels = true_labels.float()

# Define the loss function
criterion = nn.BCEWithLogitsLoss()

# Forward pass: Compute predicted logits by passing 'images' to the model
predicted_logits = model(images)

# Compute loss: Compare the output (logits) of the model with the true labels
loss = criterion(predicted_logits, true_labels)

# Print the loss
print(loss)
"""

# "cuda" only when GPUs are available.
device = "cuda" if torch.cuda.is_available() else "cpu"

# Initialize a model, and put it on the device specified.
#model = Classifier().to(device)

"""
#resnet18
model = torchvision.models.resnet18(pretrained=False).to(device)
model_weight_path = "./resnet18_pre.pth"
model.load_state_dict(torch.load(model_weight_path), strict=False)
"""

#resnet34
'''
model = torchvision.models.resnet34(pretrained=False).to(device)
model_weight_path = "./resnet34_pre.pth"
model.load_state_dict(torch.load(model_weight_path), strict=False)

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 11).to(device)
model.device = device
'''
model = ResNet50(img_channels=3, num_classes=11).to(device)  # or any other configuration you need

# fix para in model layers
#for param in model.parameters():
#    param.requires_grad = True

model.device = device

# For the classification task, we use cross-entropy as the measurement of performance.
# Not to apply softmax before CrossEntropyLoss()
criterion = nn.CrossEntropyLoss() # -log(max(softmax(output))), then compare the value with the actual data


# Initialize optimizer, you may fine-tune some hyperparameters such as learning rate on your own.
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003,betas=(0.9, 0.999),eps=1e-8, weight_decay=1e-4)

# The number of training epochs.
n_epochs = 100

# Whether to do semi-supervised learning.
do_semi = True
best_acc = 0.0
train_loss_record = []
valid_loss_record = []
train_acc_record = []
valid_acc_record = []


for epoch in range(n_epochs):
    # ---------- TODO ----------
    # In each epoch, relabel the unlabeled dataset for semi-supervised learning.
    # Then you can combine the labeled dataset and pseudo-labeled dataset for the training.
    if do_semi and best_acc > 0.62:
        # Obtain pseudo-labels for unlabeled data using trained model.
        pseudo_set = get_pseudo_labels(unlabeled_set, model)

        # Construct a new dataset and a data loader for training.
        # This is used in semi-supervised learning only.
        concat_dataset = ConcatDataset([train_set, pseudo_set])
        train_loader = DataLoader(concat_dataset, batch_size=batch_size, shuffle=True, num_workers=16, pin_memory=True, drop_last=True)

    # ---------- Training ----------
    # Make sure the model is in train mode before training.
    model.train()

    # These are used to record information in training.
    train_loss = []
    train_accs = []


    # For multi-label classification
    """
    true_labels = true_labels.float()

    # Define the loss function
    criterion = nn.BCEWithLogitsLoss()

    # Forward pass: Compute predicted logits by passing 'images' to the model
    predicted_logits = model(images)

    # Compute loss: Compare the output (logits) of the model with the true labels
    loss = criterion(predicted_logits, true_labels)
    """
    # Iterate the training set by batches.
    for batch in tqdm(train_loader):

        # A batch consists of image data and corresponding labels.
        imgs, labels = batch

        # Forward the data. (Make sure data and model are on the same device.)
        logits = model(imgs.to(device))

        # Calculate the cross-entropy loss.
        # We don't need to apply softmax before computing cross-entropy as it is done automatically.
        loss = criterion(logits, labels.to(device))

        # Gradients stored in the parameters in the previous step should be cleared out first.
        optimizer.zero_grad()

        # Compute the gradients for parameters.
        loss.backward()

        # Clip the gradient norms for stable training.
        # 避免梯度爆炸or消失，將梯度限制在某個範圍
        grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)

        # Update the parameters with computed gradients.
        optimizer.step()

        # Compute the accuracy for current batch.
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # Record the loss and accuracy.
        train_loss.append(loss.item())
        train_accs.append(acc)


    # The average loss and accuracy of the training set is the average of the recorded values.
    train_loss = sum(train_loss) / len(train_loss)
    train_acc = sum(train_accs) / len(train_accs)

    # Print the information.
    print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")

    # ---------- Validation ----------
    # Make sure the model is in eval mode so that some modules like dropout are disabled and work normally.
    model.eval()

    # These are used to record information in validation.
    valid_loss = []
    valid_accs = []

    # Iterate the validation set by batches.
    for batch in tqdm(valid_loader):

        # A batch consists of image data and corresponding labels.
        imgs, labels = batch

        # We don't need gradient in validation.
        # Using torch.no_grad() accelerates the forward process.
        with torch.no_grad():
          logits = model(imgs.to(device))
        #logits = [[2.0, 1.0, 0.5],[0.8, 1.5, 3.0]] for 2 observation with 3 class
        # We can still compute the loss (but not the gradient).
        loss = criterion(logits, labels.to(device))

        # Compute the accuracy for current batch.
        acc = ( logits.argmax(dim=-1) == labels.to(device)).float().mean()
        #Logits will return the index number for the largest value of class
        # Record the loss and accuracy.
        valid_loss.append(loss.item())
        valid_accs.append(acc)

    # The average loss and accuracy for entire validation set is the average of the recorded values.
    valid_loss = sum(valid_loss) / len(valid_loss)
    valid_acc = sum(valid_accs) / len(valid_accs)

    # Print the information.
    print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")

    if valid_acc > best_acc:
      best_acc = valid_acc
      torch.save(model.state_dict(), './model.ckpt')
      print('saving model with acc {:.5f}'.format(best_acc))

    #record for visualization
    train_loss_record.append(train_loss)
    valid_loss_record.append(valid_loss)
    train_acc_record.append(train_acc)
    valid_acc_record.append(valid_acc)

  0%|          | 0/404 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
100%|██████████| 404/404 [04:11<00:00,  1.61it/s]


[ Train | 001/100 ] loss = 1.63436, acc = 0.46148


100%|██████████| 7/7 [00:07<00:00,  1.02s/it]


[ Valid | 001/100 ] loss = 2.29555, acc = 0.31718
saving model with acc 0.31718


100%|██████████| 404/404 [04:16<00:00,  1.58it/s]


...

100%|██████████| 397/397 [04:04<00:00,  1.62it/s]


[ Train | 016/100 ] loss = 0.36420, acc = 0.88439


100%|██████████| 7/7 [00:06<00:00,  1.12it/s]


[ Valid | 016/100 ] loss = 1.34275, acc = 0.63797

New pseudo label data: 19208



  9%|▊         | 34/392 [00:28<08:22,  1.40s/it]

In [ ]:
class PseudoDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, id):
        return self.x[id][0], self.y[id]

def get_pseudo_labels(dataset, model, threshold=0.9):
    # This functions generates pseudo-labels of a dataset using given model.
    # It returns an instance of DatasetFolder containing images whose prediction confidences exceed a given threshold.
    # You are NOT allowed to use any models trained on external data for pseudo-labeling.
    # from https://github.com/1am9trash/Hung_Yi_Lee_ML_2021/blob/main/hw/hw3/hw3_code.ipynb
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Construct a data loader.
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    # Make sure the model is in eval mode.
    model.eval()
    # Define softmax function.
    softmax = nn.Softmax(dim=-1)

    idx = []
    labels = []

    # Iterate over the dataset by batches.
    for i, batch in enumerate(data_loader):
        #Image+ y_Label(As it is unlabel, so it is _)
        img, _ = batch
        with torch.no_grad(): #Disable gradient while training
            logits = model(img.to(device))
            #Suppose it have 2 class with 3 sample in the batch
            #logits=[[5,-9],[2,-3],[0.02,0.9]]
        probs = softmax(logits)
        #props=[[0.9,0.1],[0.5,0.5]] , softmax default dim=1 for torch.softmax(logits, dim=1)

        for j, x in enumerate(probs):
            if torch.max(x) > threshold:
                idx.append(i * batch_size + j)
                labels.append(int(torch.argmax(x)))

    model.train()
    print ("\nNew pseudo label data: {:5d}\n".format(len(idx)))
    dataset = PseudoDataset(Subset(dataset, idx), labels)
    return dataset

## **Testing**

For inference, we need to make sure the model is in eval mode, and the order of the dataset should not be shuffled ("shuffle=False" in test_loader).

Last but not least, don't forget to save the predictions into a single CSV file.
The format of CSV file should follow the rules mentioned in the slides.

### **WARNING -- Keep in Mind**

Cheating includes but not limited to:
1.   using testing labels,
2.   submitting results to previous Kaggle competitions,
3.   sharing predictions with others,
4.   copying codes from any creatures on Earth,
5.   asking other people to do it for you.

Any violations bring you punishments from getting a discount on the final grade to failing the course.

It is your responsibility to check whether your code violates the rules.
When citing codes from the Internet, you should know what these codes exactly do.
You will **NOT** be tolerated if you break the rule and claim you don't know what these codes do.


In [ ]:
# Make sure the model is in eval mode.
# Some modules like Dropout or BatchNorm affect if the model is in training mode.
""" turn off when using resnet
del model
model = Classifier().to(device)
model.load_state_dict(torch.load('./model.ckpt'))
"""
model.eval()

# Initialize a list to store the predictions.
predictions = []

# Iterate the testing set by batches.
for batch in tqdm(test_loader):
    # A batch consists of image data and corresponding labels.
    # But here the variable "labels" is useless since we do not have the ground-truth.
    # If printing out the labels, you will find that it is always 0.
    # This is because the wrapper (DatasetFolder) returns images and labels for each batch,
    # so we have to create fake labels to make it work normally.
    imgs, labels = batch

    # We don't need gradient in testing, and we don't even have labels to compute loss.
    # Using torch.no_grad() accelerates the forward process.(Disable gradient calculation)
    with torch.no_grad():
        logits = model(imgs.to(device))

    # Take the class with greatest logit as prediction and record it.
    predictions.extend(logits.argmax(dim=-1).cpu().numpy().tolist())

In [ ]:
# Save predictions into the file.
with open("predict.csv", "w") as f:

    # The first row must be "Id, Category"
    f.write("Id,Category\n")

    # For the rest of the rows, each image id corresponds to a predicted class.
    for i, pred in  enumerate(predictions):
         f.write(f"{i},{pred}\n")

## **Visualize Result**



In [ ]:
import matplotlib.pyplot as plt

x = np.arange(len(train_acc_record))
train = torch.tensor(train_acc_record, device='cpu')
valid = torch.tensor(valid_acc_record, device='cpu')

plt.xlabel("epoch")
plt.ylabel("acc")
plt.plot(x, train, color="blue", label="Train")
plt.plot(x, valid, color="red", label="Valid")
plt.legend(loc="best")
plt.show()

In [ ]:
import matplotlib.pyplot as plt

x = np.arange(len(train_loss_record))
train = torch.tensor(train_loss_record, device='cpu')
valid = torch.tensor(valid_loss_record, device='cpu')

plt.plot(x, train_loss_record, color="blue", label="Train lost")
plt.plot(x, valid_loss_record, color="red", label="Valid lost")
plt.legend(loc="best")
plt.show()